In [42]:
#! pip install -r requirements.txt
import pandas as pd, numpy as np
import glob, re,json,io,os
from datetime import datetime, timedelta
import plotly.graph_objects as go
pd.options.display.max_colwidth = 100
from numpy.random import random
from numpy.random import choice
rng = np.random.default_rng()
from google.cloud import bigquery,storage
project_id = 'dett-stt'
clientB = bigquery.Client(project_id)
storageC=storage.Client(project_id)


In [ ]:
#@title Lecture des issues et fermetures et génération des horodates de visites d'issues programmées
issues=pd.read_csv('https://exploitidf.github.io/IssuesTunnels/_downloads/caee2520ef1721f8437118adc2284ddf/issuesFermetures.csv')[[  'Tatouage',   'Fermeture']]
fermetures=issues['Fermeture'].value_counts()
lenFer=len(fermetures )
VisiteIssues=[]
for k in range(6):
  jours=pd.Series([(5+k*8)*7+ int(random()*4)*7+int(random()*4) for i in range(lenFer)],index=fermetures.index,name='jour')
  fermTmp=pd.concat([fermetures,jours],axis=1)
  isInd=issues.join(jours,on='Fermeture')
  isInd.index=[k*1000+j for j in issues.index]
  isInd['OT']=[345467+13000*k+m for m in issues.index]
  isInd['ordreVB']=k
  VisiteIssues.append(isInd)
VisiteIssues=pd.concat(VisiteIssues)
minutes=pd.Series([int(random()*240) for i in range(len(VisiteIssues))],index=VisiteIssues.index,name='minute')
VisiteIssues=pd.concat([VisiteIssues,minutes],axis=1)
VisiteIssues['HoroDate']=pd.to_datetime("24/"+VisiteIssues['jour'].astype(str)+" 23",format='%y/%j %H')+pd.to_timedelta(VisiteIssues['minute']*60000000000)
VisiteIssues=VisiteIssues[['Fermeture','ordreVB', 'OT','Tatouage', 'HoroDate']].reset_index(drop=True)


In [29]:
# Lecture du tableau des points de contrôle initial. Ce format devrait changer ...
vbIa=pd.read_csv('../_static/controleVB_IA.csv',encoding='UTF-8').iloc[:,1:]
lstPC=vbIa['PointControle'].value_counts().reset_index()
lstPC['ordrePC']=lstPC['PointControle'].str.split('.').apply(lambda x: x[0]).astype(int).astype(str).str.zfill(2) 
lstPC=lstPC.sort_values('ordrePC').reset_index(drop=True) #.set_index('ordrePC',drop=True)
lstLst=[]
for i in range(len(lstPC)) :
  vbIi=vbIa[vbIa['PointControle']==lstPC.loc[i,'PointControle']]
  lstCom=[]
  for j in vbIi.index:
    lstCom.append(vbIi.loc[j,'Note'][0]+'_'+vbIi.loc[j,'ResulControle'])
  lstLst.append(sorted(lstCom, reverse=False))

lstNbrResul=[len(ll) for ll in lstLst]
lsP=[[round(.2/(i-1),3)]*(i-1) +[.8]    for i in lstNbrResul]
lstLstR=[['R'+str(i) for i in range(j)] for j in lstNbrResul ]

tabRC=[]
for i in range(len(lstPC)) :
  for j in range(lstNbrResul[i]) :
    tabRC.append(['P'+ lstPC.loc[i,'ordrePC'], lstPC.loc[i,'PointControle'],lstLstR[i][j],lstLst[i][j],lsP[i][j]  ] )
pd.DataFrame(tabRC,columns=['codePC','PC','codeRC', 'RC', 'Probab']).to_csv('controleVB_IS.csv',index=False)
# Dans le futur, on devrait entrer directement une table de ce type.


In [ ]:
def simuRapp(ferm,j):
    vst=VisiteIssues[(VisiteIssues['Fermeture']==ferm)&(VisiteIssues['ordreVB']==j)].reset_index(drop=True)  
    for k in range(len(vst)):
        dateFichier= str(int((vst.loc[k,'HoroDate']-datetime(2024,1,1)).total_seconds()+1000*random()))
        flNm='24-3/ISVB-'+ str(vst.loc[k,'OT'])+'-'+dateFichier+ '.json'
        with open(flNm,'w') as fl:
            fl.write('{')
            fl.write('"Tatouage":"'+str(VisiteIssues.loc[k,'Tatouage'])+'",')
            fl.write('"HoroDate":"'+str(VisiteIssues.loc[k,'HoroDate'])+'",')
            fl.write('"Agent":"'+choice(['Karl','Karen', 'Kim','Kamel','Kun']) +'",')
            fl.write('"PC'+str(0)+'":"'+choice(a=lstLstR[0])+'"')
            for n in range(1,len(lstPC)):
                fl.write(' ,"PC'+str(n)+'":"'+choice(a=lstLstR[n])+'"') 
                fl.write('}')


In [ ]:
# Création des rapports avec une probabilité variable par fermeture et par issue
if False :
    for ferm in list(fermetures.index):
        rndF=round((random())**.2,3)*4
        for j in range(6):
            if (rndF+(random()**5))>3:
                simuRapp(ferm,j)

In [39]:
fermetures
ll=[]
for i in range(999):
    ll.append(round((random())**.5,3)*1000)
pd.Series(ll).value_counts().iloc[:3]

862.0    6
874.0    6
924.0    6
Name: count, dtype: int64

In [ ]:
filenames = next(os.walk('24-3'), (None, None, []))[2]
len(filenames)


['ISVB-345467-4840330.json',
 'ISVB-345467-5012025.json',
 'ISVB-345468-4841846.json']

In [78]:
# https://storage.googleapis.com/issues-secours/rapports-visites/345512-3207090.json

blobs = storageC.list_blobs(bucket)
for blob in blobs:
    if 'ISV' in blob.name:
        blob.delete()

from time import sleep
bucket = storageC.get_bucket('issues-secours')
for fl in filenames[:10]:
    blob = bucket.blob('rapports-visites/'+fl)
    blob.upload_from_filename('24-3/'+ fl, if_generation_match= 0)
    sleep(3)





In [73]:
blobs = storageC.list_blobs(bucket)
for blob in blobs:
    if 'ISV' in blob.name:
        print(blob.name)
        blob.delete()

rapports-visites/ISVB-345467-4840330.json
rapports-visites/ISVB-345467-5012025.json
rapports-visites/ISVB-345468-4841846.json
rapports-visites/ISVB-345468-5012845.json
rapports-visites/ISVB-345469-4839847.json
rapports-visites/ISVB-345469-5008993.json
rapports-visites/ISVB-345470-4842791.json
rapports-visites/ISVB-345470-5008322.json
rapports-visites/ISVB-345471-4838914.json
rapports-visites/ISVB-345471-5011273.json
rapports-visites/ISVB-345472-4840063.json
rapports-visites/ISVB-345472-5012302.json
rapports-visites/ISVB-345473-4836587.json
rapports-visites/ISVB-345473-5011539.json
rapports-visites/ISVB-345474-4835922.json
rapports-visites/ISVB-345474-5014200.json
rapports-visites/ISVB-345475-4849427.json
rapports-visites/ISVB-345475-5015140.json
rapports-visites/ISVB-345476-4842508.json
rapports-visites/ISVB-345476-5019667.json
